In [3]:
from mingpt.model import GPT, GPTConfig
import torch

model_path = "29_12_2021_02_55_28.model"

block_size = 128

from mingpt.char_dataset import CharDataset
text = open('input.txt', 'r', encoding="utf8").read()
train_dataset = CharDataset(text, block_size)

mconf = GPTConfig(train_dataset.vocab_size, block_size, n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)
model.load_state_dict(torch.load(model_path))

data has 1170335 characters, 3567 unique.


<All keys matched successfully>

In [7]:
batch_size = 128
num_workers = 0

In [8]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=batch_size, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=num_workers)
trainer = Trainer(model, train_dataset, None, tconf)

Using GPU 0


In [10]:
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample

context = "The quick brown fox jumps over the lazy dog"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 200, temperature=1.0, sample=True, top_k=1)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])

print(completion)

冰滴咖啡用左京都流行嘅感覺 最後教我地先飲一啖咖啡，覺得苦澀，可以含一糖一齊飲，跟住飲啖水，清一清口腔嘅異味，再去飲下一啖咖啡，咁先可以真真正正咁品嘗到咖啡嘅香味。老老實實，我真飲唔出咖啡同出面嘅有好大分別，少少苦味，但有好味嘅糖可以一齊送咖啡，我覺得都可以接受，咖啡香味的確好濃，我個飲慣咖啡嘅朋友，都話好香好正。最得意嘅係，飲完咖啡，店主話可以將個咖啡杯倒轉，然後佢可以研究杯入面咖啡漬嘅pattern黎算命，200蚊
